# importing libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

In [ ]:
df= pd.read_csv('/kaggle/input/fashion-products/fashion_products.csv')

# EDA

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.shape

# Distribution of Categorical Variables

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Category')
plt.title('Count of Products in Each Category')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
df['Brand'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Brands')
plt.ylabel('')
plt.show()

In [ ]:
sns.set_style('whitegrid')
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot distribution of prices
sns.histplot(data=df, x='Price', bins=10, ax=axes[0])
axes[0].set_title('Distribution of Prices')

# Plot distribution of ratings
sns.histplot(data=df, x='Rating', bins=10, ax=axes[1])
axes[1].set_title('Distribution of Ratings')
plt.tight_layout()
plt.show()

# Multivariate Analysis


In [ ]:
plt.figure(figsize=(8, 6))
category_brand_counts = df.groupby(['Category', 'Brand']).size().unstack()
sns.heatmap(data=category_brand_counts, cmap='Blues', annot=True, fmt='g')
plt.title('Product Count by Category and Brand')
plt.show()

plt.figure(figsize=(8, 6))
sns.pairplot(df[['Price', 'Rating', 'Category']], hue='Category')
plt.title('Pairwise Relationships')
plt.show()


# Hybrid Recommendation System

In [ ]:
# Create the Surprise dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['User ID', 'Brand', 'Rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Collaborative Filtering using SVD
algo_cf = SVD()
algo_cf.fit(trainset)

# Brand-Based Recommendation System
def brand_recommendation(user_id, num_recommendations):
    # Collaborative Filtering predictions
    cf_predictions = algo_cf.test(testset)

    # Filter predictions based on user ID and brand
    user_brand_predictions = [pred for pred in cf_predictions if pred.uid == user_id and pred.iid in df[df['User ID'] == user_id]['Brand'].unique()]

    # Sort the predictions by rating in descending order
    user_brand_predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top N recommendations
    top_recommendations = user_brand_predictions[:num_recommendations]

    return top_recommendations

# Example usage: Get brand recommendations for User ID 97, top 3 recommendations
user_id = 6
num_recommendations = 3
recommendations = brand_recommendation(user_id, num_recommendations)

# Print the recommendations
print(f"Brand Recommendations for User ID {user_id}:")
for recommendation in recommendations:
    brand = recommendation.iid
    rating = recommendation.est
    print(f"Brand: {brand}, Rating: {rating}")